## RLAIF Example - Finetuning with Sagemaker

This notebook demonstrates basic user flow for RLAIF Finetuning from a model available in Sagemaker Jumpstart.
Information on available models on jumpstart: https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-latest.html

### Setup and Configuration

Initialize the environment by importing necessary libraries and configuring AWS credentials

In [1]:
#!/usr/bin/env python3

from sagemaker.train.rlaif_trainer import RLAIFTrainer
from sagemaker.train.configs import InputData
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage
import os
#os.environ['SAGEMAKER_REGION'] = 'us-east-1'
#os.environ['SAGEMAKER_STAGE'] = 'prod'

import boto3
from sagemaker.core.helper.session_helper import Session

# For MLFlow native metrics in Trainer wait, run below line with approriate region
os.environ["SAGEMAKER_MLFLOW_CUSTOM_ENDPOINT"] = "https://mlflow.sagemaker.us-west-2.app.aws"



[12/02/25 10:24:29] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=932969;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=642938;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=777241;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=529130;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

### Create RLAIFTrainer
**Required Parameters** 

* `model`: base_model id on Sagemaker Hubcontent that is available to finetune (or) ModelPackage artifacts

**Optional Parameters**
* `reward_model_id`: Bedrock model id, supported evaluation models: https://docs.aws.amazon.com/bedrock/latest/userguide/evaluation-judge.html
*  `reward_prompt`: Reward prompt ARN or builtin prompts refer: https://docs.aws.amazon.com/bedrock/latest/userguide/model-evaluation-metrics.html
* `model_package_group_name`: ModelPackage group name or ModelPackageGroup
* `mlflow_resource_arn`: MLFlow app ARN to track the training job
* `mlflow_experiment_name`: MLFlow app experiment name(str)
* `mlflow_run_name`: MLFlow app run name(str)
* `training_dataset`: Training Dataset - either Dataset ARN or S3 Path of the dataset (Please note these are required for a training job to run, can be either provided via Trainer or .train())
* `validation_dataset`: Validation Dataset - either Dataset ARN or S3 Path of the dataset
* `s3_output_path`: S3 path for the trained model artifacts

In [3]:
# For fine-tuning 
rlaif_trainer = RLAIFTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="sdk-test-finetuned-models", # Make it Optional
    reward_model_id='anthropic.claude-3-5-sonnet-20240620-v1:0',
    reward_prompt='Builtin.Correctness',
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlaif-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlaif-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    accept_eula=True
    #sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/Admin"
)

[12/01/25 12:12:18] INFO     SageMaker session not provided. Using default Session.                  ]8;id=126215;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=479582;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

### Discover and update Finetuning options

Each of the technique and model has overridable hyperparameters that can be finetuned by the user.

In [4]:
print("Default Finetuning Options:")
pprint(rlaif_trainer.hyperparameters.to_dict()) # rename as hyperparameters

#set options
rlaif_trainer.hyperparameters.get_info()


Default Finetuning Options:


{
│   'data_path': 'None',
│   'global_batch_size': '128',
│   'judge_model_id': 'bedrock/openai.gpt-oss-120b-1:0',
│   'judge_prompt_template': '/opt/ml/code/verl/summarize.jinja',
│   'learning_rate': '1e-05',
│   'max_epochs': '2',
│   'max_prompt_length': '1024',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   'name': 'example-name-c9jrd',
│   'output_path': '/opt/ml/model',
│   'results_directory': '',
│   'resume_from_path': '',
│   'rollout': '8',
│   'train_val_split_ratio': '0.9',
│   'validation_data_path': 'None'
}


data_path:
  Current value: None
  Type: string
  Default: None
  Required: Yes

global_batch_size:
  Current value: 128
  Type: integer
  Default: 128
  Valid options: [128, 256, 512, 1024]
  Required: Yes

judge_model_id:
  Current value: bedrock/openai.gpt-oss-120b-1:0
  Type: string
  Default: bedrock/openai.gpt-oss-120b-1:0
  Required: Yes

judge_prompt_template:
  Current value: /opt/ml/code/verl/summarize.jinja
  Type: string
  Default: /opt/ml/code/verl/summarize.jinja
  Valid options: ['/opt/ml/code/verl/cot.jinja', '/opt/ml/code/verl/evaluate.jinja', '/opt/ml/code/verl/faithfulness.jinja', '/opt/ml/code/verl/summarize.jinja']

learning_rate:
  Current value: 1e-05
  Type: float
  Default: 1e-05
  Range: 1e-07 - 0.001
  Required: Yes

max_epochs:
  Current value: 2
  Type: integer
  Default: 2
  Range: 1 - 30
  Required: Yes

max_prompt_length:
  Current value: 1024
  Type: integer
  Default: 1024
  Range: 512 - 16384
  Required: Yes

mlflow_run_id:
  Current value: 
  Type: 

#### Start RLAIF training


In [4]:

training_job = rlaif_trainer.train(wait=True)


╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251123173910     │
│  MLFlow URL            ]8;id=390885;https://t-eq86xu3xab1s.us-west-2.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IjNiNWQyYmIxLWRkZmUtNDk0OC04MDA0LWI5MjRiYzA1ZjU4NyIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNHhZc2dLdFhjUFBseFhHcUpaMkROYi9HZnpJUUY2RElCaDhLK0dBNjluZDBBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVGdGIzQkdXaTlxYUVnemNVMW1aalZzVUV4b056aGFPVWhzZEZWM2VsWXhjazlrVWpkRGJWVlBlVTVNTTBGclREWkJSVGcxYkhsWWEwVlhZbkpVWlZSVVp6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzFPVEF4T0RNM016azFNRFE2YTJWNUx6ZzNabUUxTVdReUxURTRNRGt0TkdVMFl5MWhObVV6TFRRNFpXWTNNelk1WW1NM1lnQzRBUUlCQUhndFR2ZzEyNVZtTm50WE4vTEVZV1dzREMrSk1KRzNQSUl1eU9PYTB5SU5FZ0ZLdTAyc1B4bk1qVCs1UE50d1lEeUpBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU1aOG9NYUFXTmw3UzhONXJjQWdFUWdEdGVhN2toLzQ1aGNiWEhBQ0IvQnRPUFlyaUZCSU14cVBkVzJuWmJocWJ4c2FWTzYxUW5tQ1lISFU3K1lnOE1BZjEwMDVPTXYydTcySVE5K3dJQUFCQUFKZWxhQVNWWGJySTN3cmVnRDhYTDI5ZVphc1NaRTQzVXBjcXc1RWJsOFJtSS91Tk5KckFvUHdpaFFSdzhmcHg2Ly8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBUkhJTFhiaGNQMHVscU9abytQak9wUlo3ZFZQZy9ZRk9VMVNzRWlwbzhUeFo0WlhHb1VHVWNDRkZDRldBVHdXWWtLOWJ0VEx3RG9aRWlCNkhmenU4aWV6MnZBcVZwMFEwaDZLanNiWG1yVy9LL0lFSkhGQzgybFp3cFZ6WnpIc2FRdjlmVTE3aVJodTlaZVpxZk1xM1Y0MG1LRkNVRW5HdnZHZWlFWVNVOFRoc3VoclRzRG1SN01qZnBWeWs4dGhiWG5DWmNrVnJyK1UrbFZnbERpVGRBUm1zYTdob1ZpVWl1L2NhejRMY0MrUGswVFVzZFlzNkJiTWtpV1RZeGVFSEViamt2QUNqckFZQ3ljN3JhVFZDUlJzM3d3MEcxMHFzMlgvaW8wRjAwR0NoZFI3WUlvaE1XWnk5UlA5eXl6Tnd2cWd4Qk5EcVpKS2pTYktvZFBFVGpBdFJ0YU5sRlNPbHJuelNxcWdrZmJHQXN1ZzlIRFlVTGQxR3JndXNOTllWRW95Z1R4K09QQStyTFZsVkhEUUdJM0JBdDF6UzhkTTNmWXplbGhndUpjNHdYZlFwYTJOeWtpYkZpVlpxSnF0b0ZpRzJBUTc3clh0VXRDR1NFZGRFRHVIZlNSTVZLMW56Smp2L0dUSm1LRGhLU01maGFKNFg3SE5Sd1Bmd1AzR1hHRmRmeVBrYm5mNjUrZjVjdFY3d3V6WENtUVdaWEVjbUxsYmhmdUJ1YnhuaWhYMW9zQ1dLUmpjbVJXQ2RES1NaOVd4L2UxQ2V2bzNkK2xOMFpwRlpEaXFzRFBBNDM4ODk3NlE5ZGtOSUUwbzFvTk9tUHF2RGVqZzRjWDRUVDdTeGtkdkZFVEVtMjFpOHNmMnl4ekVwRGJOTWZPUkNabVFoTTlyMWc4TllwUTcwQ1BEd1hJMkQweU95Tnlsc2hTOGFrRDNpVHBhb0htc2x4VUhCT0UyM0VOSG1VbzFidDRHZmRzS2NqQ1FzOTE5ZnROWmJ2UXBuYWRDL0U4akcwVW1zU2Q0UGlHRnVLSG9XbldSeFZqM2xXaVFKYWt5NVVpOTZvQXR3OU1RVmVCWVpSM25PdVlUb05OWlhIcE9JR3gwbGczVEVsSmZ3ZzNIVmwyRnlSQk9sRUNPdnlxMWh5Z2FaOWUvZjBWTFNLQXJTTFdvMGcxNVoyWDZyaUhRZGhHRks1RG5YTDA5ZlBuR1Y5UjZMTDFyMUtxZkJFN1N0OVYwd25KTjkrcERDTzhKVGdUS01KSG9Xei96UnVvTHJwdVhiU1E5dnRuZCsvUUMwNDhSRGxrdTQ2UEdCY2VpV3pMcE41dlpMTG9rSkJWNDV3c2Q4MCt4Y0VkaDUxU0Z6OEZFZm80cE5IK2xLdVJpbXl2enFoY3NRRXpwcjZtMGx6Q2tSWVhxRDJMRWNDQ1IrdlRyR3BiemtXdlVnSi9UbDZ6THR6VTlpRVBLbWhTckdENEZjYUdSajc1aUhZbEU3ZDlnYVJSdDV3OEZYYS8xWms0TkdBSlFXenI2dkc3TGIxV2V5SHU3YTAwcDFsRUlMMjlRcXpzb0hSUzRDaWtLK0xaQlZTd3NRVW1xQ2U4dlFoWmRHNFhsZWFDM2NYMkxQRnBaUVpJZmdmL2ZxdXI1M0JMbVByVWdreTdPM21xVlNTZ0hWZkRQTldLeFBYZUl4RmlSWEZNZng1dGRIdWJNZUxEUEh2enFOdXYxU2Y0VXBGb3JDQjZWWWdhK0FiU0xjaWs0TjNNZm9yRk0vN1dqekV5NE9TYWVRNzR2NFVDblJoVDFKMG5yeHBBSnpVQlNkanN0N0pqc054NVdKWFNzV1pUblFUWHZHMktURFVwb3dzSkh3K0ZKRnYxaVZEUVluczl4WWMxTzFlNlVqNm16QjFPOHY1WGxzV215eWpkM2J1ajc5ekJRbldzcUZ2d0RwWEFta04zOE00NGxSNENwZ3d6anRoUUhOamJnQWNHUmdCbk1HVUNNUUN4NlZTTXZlTFpRNDF4UjIzeTk2OERlUDFoNWNCZmVCWWtYaXVrRDRTNkJKTGJmQUdpZWU4RUNDc3E4dE5LT3pVQ01ESGY3SUxEbFlld3hEWEF5aTFwanVqUzZDdWkvTEhxQ2Z2T0VrRlM3S1dHMFdmWUlDTHY5bHErcUkvdmFnbjl2QT09IiwiY2lwaGVyVGV4dCI6IkFRSUJBSGd0VHZnMTI1Vm1ObnRYTi9MRVlXV3NEQytKTUpHM1BJSXV5T09hMHlJTkVnR0xDVHh6VHBZQ2xUTUI0L1c0ZGRiVEFBQUFvakNCbndZSktvWklodmNOQVFjR29JR1JNSUdPQWdFQU1JR0lCZ2txaGtpRzl3MEJCd0V3SGdZSllJWklBV1VEQkFFdU1CRUVEUGVTeWUyOFRWVnJWOEdhamdJQkVJQmIvT3JFMHJIeDRrandRS1QzL1VQdEFHeXhOcjAzenl4blE1NUJmelBRRlppTWp6TDEzdFZQWXVBQ0pqVTM5dEtGN2NJeFNqd1FySHMvRXFvWHRHY0xzR3ZYTmhqQmpkMHkvUnVvK2FCMXZQamFreTU1Njl0VEtuR0VNQT09Iiwic3ViIjoiYXJuOmF3czpzYWdlbWFrZXI6dXMtd2VzdC0yOjA1MjE1MDEwNjc1NjptbGZsb3ctdHJhY2tpbmctc2VydmVyL21tbHUtZXZhbC1leHBlcmltZW50IiwiaWF0IjoxNzYzOTQ4

In [5]:
training_job = rlaif_trainer.train(wait=True)


╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251201121238    │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          614.6s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             0.7s                      │
│   ✓    Pending           Preparing the instances for           15.7s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        5.7s                      │
│   ✓    Training          Training image download completed.    551.9s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    38.7s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251201121238',
│   training_job_arn='arn:aws:sagemaker:us-west-2:729646638167:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251201121238',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251201121238/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251201121238/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '128',
│   │   'judge_model_id': 'bedrock/openai.gpt-oss-120b-1:0',
│   │   'judge_prompt_template': '/opt/ml/code/verl/summarize.jinja',
│   │   'learning_rate': '1e-05',
│   │   'max_epochs': '2',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-c9jrd',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '8',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   role_arn='arn:aws:iam::729646638167:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://mc-flows-sdk-testing/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1173048f0>,
│   │   max_pending_time_in_seconds=<

### View any Training job details

We can get any training job details and its status with TrainingJob.get(...)

In [28]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251124140754")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251124140754',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251124140754',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251124140754/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251124140754/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'judge_model_id': 'bedrock/openai.gpt-oss-120b-1:0',
│   │   'judge_prompt_template': '/opt/ml/code/verl/summarize.jinja',
│   │   'learning_rate': '1e-05',
│   │   'lora_alpha': '256',
│   │   'max_epochs': '2',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-ea0mx',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '8',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   capacity_schedules_config=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   training_plan_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>,
│   │   instance_placement_config=<sagemaker.core.utils.utils.Unassigned object at 0x11446b7d0>
│   ),
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object 

### Test RLAIF with Custom Reward Prompt

Here we are providing a user-defined reward prompt/evaluator ARN

In [2]:


# For fine-tuning 
rlaif_trainer = RLAIFTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="sdk-test-finetuned-models", # Make it Optional
    reward_model_id='anthropic.claude-3-5-sonnet-20240620-v1:0',
    reward_prompt="arn:aws:sagemaker:us-west-2:729646638167:hub-content/sdktest/JsonDoc/rlaif-test-prompt/0.0.1",
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlaif-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlaif-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl", #Optional[str]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    accept_eula=True
    #sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    #role="arn:aws:iam::052150106756:role/Admin"
)


[12/02/25 10:28:14] INFO     SageMaker session not provided. Using default Session.                  ]8;id=423696;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=732501;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Runs on sagemaker prod, region:us-west-2                                  ]8;id=836889;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=831071;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

In [3]:
training_job = rlaif_trainer.train(wait=True,
                                   logs=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251119221710    │
│  MLFlow URL            ]8;id=595913;https://t-eq86xu3xab1s.us-west-2.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IjUzNjQxYzc2LTUwYjYtNDM1YS05ZTI4LTY1OGMwZTZlM2ZjOSIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNGYvd3NIT0FlMDZMU2NmL09vYUZSMGx6MTlyTU5BM3o1UUpMaG1NMmFQemdBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVFclVHRmpjell6VW05M1VYWnhabkZrYURObVJFaHhNM1ZEUjNWYWRtdHpWMDV0YVVKdmJYaFhWMnN4TDNoblExbHFiVm80VkZvM2JYcEtNRTVaT1RGRVp6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzFPVEF4T0RNM016azFNRFE2YTJWNUx6ZzNabUUxTVdReUxURTRNRGt0TkdVMFl5MWhObVV6TFRRNFpXWTNNelk1WW1NM1lnQzRBUUlCQUhndFR2ZzEyNVZtTm50WE4vTEVZV1dzREMrSk1KRzNQSUl1eU9PYTB5SU5FZ0hLcGRFL1hPbzYxYnFuNUJNK1FVVk1BQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU1LNlNjWjJNRjJJSitrb2hRQWdFUWdEdFAreTUvTjl0Z3ZVMUVYUHFUSjZFMENwRENJZmluUVhMOW04QUhNYS9qRWgzbTFLZVRyWUNUSGZFVVJjaTBEVVF4ZWJJSTdUdUF3dGhPb3dJQUFCQUFmS2ZlTnlQWTVtYUxZZWNrVytZMTVKbVg2TXlJbkN3bkFiQjhwak9HeWVoN2hPRDhGSkh3QXlwalJ3TS92Sk14Ly8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBUTdiMkJXT29XL0NpdVVwZ2UvaHV3Z0pSUGRHYlprRk5Cbml5ZzJueUVsaXRCa1I3Q3M3a04xUXdOMkp6Vm5PTzdDNWtTdk9OTVB4SnYrN3pMNlZDR2FXV1Z3V1hUMHBBSU5DOHRjZlYzQVVQbUlnSkhGMi9oWkEvbmxvaldNMjBkM0NsQUxXWFZsNnJnNW93SXJzek4vOExLSm9ZUUMrTXYrVTRSQzVYckpYWXB0K3hpc3AxSG5iU3pFMG03R0tQcGZrVHpFL2FiSElwNCtqSUlJOE5YNEo5Um9BNnhkbjhPb0ZwOXFpVVlrTjljOWkwZ3lIa0xTQTNxMWxURklXazgvbHZZM3JGVGhJZnNpWFo3OE9QQk5mTkxrT0hvSUZmK2ZNemdVRUp4c0NIeVdENC9IN1dyWjJ2WC9IWHN1NUI1ajQ5N0RUQ3R0NGRRQWVhbWQxeGZVaUVsaGRvSWNsVWZMeHZxRGxqMmdKYm9qazRHQTExalFINmwvUEFvQU5XT3dZMWd4OVdqbStFVFp2dVdYOVpZSjhxVFNISkQxZDVRam96NW1GbDFBaXRyVjZJZnIzMDRxQk9qVXpIVTdmdks1RW9jTE92U2U5NHFaNVpEbGdvLzhkWDVuNTU0ZzlzdjJKLzEzR015SDBxa3VZNVU0eUpRQ3c5NExCRURnTVBBL1N3WU1aUTBwa1NCU2pLbFBoQ0tEKzB5RDVWNGNzcVF2dlYvN3NNRStybUJnWm84emJENGw3NmNhTWxXZzN5MmpLMWhzSWJLTE9QbWNUY0NvbHpuM29ZNExSOVN0Qlg4NVF3dFNjd29PS05SR215TjZXRXUvNUxBRkxSWFppZjl6UFpVelV6Z0VBc21iVytoRzQrT2dOaUxGajJjTXIrWG00bjN4eVlzUzJrMmhQN0FZV2lHTjEyZG9WSEhkVlNCMmZKQ2ZTdkpSTUpQMUNtV1FHNk1vNWhaMURrczlmVUFCVWFZdEtFUUFjUVltNisyWFIxWnl5d212QVM5bGJEdllhdWNWZUlYV1BCaHhHSzlqZk1XT01hM0pSVFNrVE1GQzlCMTV0MGovWkx4aERqRmRIeWhwM01OUlJJQVlOdFlqRUJvajdlQUJHRkdwVnp4Wk5aOW5qZnRlQjdJRCtiSlF2L3EyWlBOTE9kZ1pPR2Q0bkRnMkQyUndaWWxWV0hlckRpYWVNb1gzTFEybEs5OE43QldENWl5dWw3b0U4U2pla1BraDUxK3U0MkZXQ3ZVeW5VK2ZCSC96NkhuUWRvMVFrKzh1SXNFb0hFTW4rdExGUFBQTS9vQTBaQW1iN0hJT3VpcU5acWVqcllzNXZiK1Z5c1M1ZVJ1MjhUeUVabXdIbGlpL3pDZzllY3N2VFFLdis3UHZ3amh4VWp5UERXbmlUa21PVFJ0TjV1dHJEY2h4VFZRbmNVT1NTYVBGV0F0VkxuaFYydzRjdW82VGt6UmNVUlNqK1htNHhvcS8yQ0VhbWRVemg0MUwvMW1aNzFpNVc4aFZHeW00WEl0SDNJcUJueTQ4KzFGcC9jclRseVplQXVMM2dmTUxqSFp4dzdNRG90OUNLVDI0UzI0SHRJTEtiN1dyb3Vnb1JQYTJIVkRuWkhRMlZmOUdJWUwxODk3UmMreWVVSGN1TnBPVHFHRmkrZkpBK0grU1pOeGNBdUluL2dCUno0QThFdHYyNnZtaitNL214TUZWSEYwVmJ4U3lOTkxVRTJWZU91YUJHbTRQblpCYUhtb2t4eHRDbTZ1enJNQng1Wko0ZW8yaE9Ya3IwV29vWXFKS2RrY29mZ3pvUUExTmxyY0JTR2x4SVR6VlRQUVpjY0RNbU5ScUxlR2ZNa1NqcHF2S21JRTVzOHFEVnRLSVlHVnNiTDdNSTVZNUV6ZGlyTVROczRaZzdYUGF2QXNqTzRndEwvYm5Pd2w1Ty9ibTZpQ1JPQUJuTUdVQ01GNCtYc3NYaHVBbStUTE5OdWU0akpZZTZSZzFoYlQ0SzZwYVdyNFNFbzRITHdYeWtYMUpRQXBFazc0OHpNbUlWQUl4QU1BUXZyTXovaE9aelQ4enI4cUtXNnFNUzhoa0lXaDU5NC9wUmErdGFXazluMEhPaC8wRTBxNDRyMHFOMjRtd3JRPT0iLCJjaXBoZXJUZXh0IjoiQVFJQkFIZ3RUdmcxMjVWbU5udFhOL0xFWVdXc0RDK0pNSkczUElJdXlPT2EweUlORWdGRWVVbVdSQzJ1YmY3b2lBUWVlWWdiQUFBQW9qQ0Jud1lKS29aSWh2Y05BUWNHb0lHUk1JR09BZ0VBTUlHSUJna3Foa2lHOXcwQkJ3RXdIZ1lKWUlaSUFXVURCQUV1TUJFRURDUXRnckc1WlNOMFlHMC96Z0lCRUlCYmJQdkc2dThHZ1g2ZzRpSXZ0UE1MckhyZW5QMHE3b1VyTHZkcytvUXV0YjVRcE9PQVNUMzdFRE1Wc09ZZGFUeDhNTHBtRXBPNjZ4M3FoOEJaMFBxT2dEYmlOcGxTUUpPWmtmOXVFTG13V0xZY0tmV2NaVzFrMjRXUkVBPT0iLCJzdWIiOiJhcm46YXdzOnNhZ2VtYWtlcjp1cy13ZXN0LTI6MDUyMTUwMTA2NzU2Om1sZmxvdy10cmFja2luZy1zZXJ2ZXIvbW1sdS1ldmFsLWV4cGVyaW1lbnQiLCJpYXQiOjE3NjM2MTk0MzEsImV4cCI6MTc2MzYx

In [3]:
training_job = rlaif_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlaif-20251202102827    │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          801.2s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             1.4s                      │
│   ✓    Pending           Preparing the instances for           16.4s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        15.7s                     │
│   ✓    Training          Training image download completed.    722.6s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    43.7s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
pprint(training_job)

TrainingJob(
│   training_job_name='test-llama-3-2-1b-instruct-rlaif-20251111133638',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/test-llama-3-2-1b-instruct-rlaif-20251111133638',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/test-llama-3-2-1b-instruct-rlaif-20251111133638/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/test-llama-3-2-1b-instruct-rlaif-20251111133638/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   hyper_parameters={
│   │   'custom_prompt_arn': 'arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/JsonDoc/jamjee-test-rp1/0.0.1',
│   │   'data_path': 'None',
│   │   'global_batch_size': '128',
│   │   'judge_prompt_template': '/opt/ml/input/data/judge_prompt/custom_prompt.jinja',
│   │   'learning_rate': '1e-05',
│   │   'lora_alpha': '256',
│   │   'lora_rank': '128',
│   │   'max_epochs': '3',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-pjf9v',
│   │   'preset_reward_function': 'compute_score',
│   │   'reward_model_id': 'bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0',
│   │   'rollout': '8',
│   │   'temperature': '1.0',
│   │   'top_k': '-1',
│   │   'top_p': '1',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=AlgorithmSpecification(
│   │   training_input_mode='File',
│   │   training_image='',
│   │   algorithm_name=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   metric_definitions=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   enable_sage_maker_metrics_time_series=False,
│   │   container_entrypoint=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   container_arguments=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   training_image_config=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>
│   ),
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x12c04d7d0>
│   ),
│   resource_config=ResourceConfig(
│   │   volume_size_in_gb=10,
│   │   instance_type='ml.p4de.24xlarge',
│   │   instance_count=1,
│   │   volume_kms_key_id=<sagemaker